In [1]:
%%writefile requirements.txt
mlflow==1.12.1
boto3
awscli

Overwriting requirements.txt


In [2]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 13.9 MB 707 kB/s eta 0:00:01    |███▏                            | 1.4 MB 576 kB/s eta 0:00:22     |██████▊                         | 2.9 MB 680 kB/s eta 0:00:17
     |████████████████████████████████| 129 kB 375 kB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 285 kB/s eta 0:00:01
     |████████████████████████████████| 328 kB 969 kB/s eta 0:00:01
     |████████████████████████████████| 42 kB 412 kB/s eta 0:00:01
     |████████████████████████████████| 54 kB 727 kB/s eta 0:00:01
     |████████████████████████████████| 146 kB 727 kB/s eta 0:00:01
     |████████████████████████████████| 94 kB 564 kB/s eta 0:00:01
     |████████████████████████████████| 77 kB 732 kB/s eta 0:00:01
     |████████████████████████████████| 7.0 MB 708 kB/s eta 0:00:01     |███████████████████████         | 5.0 MB 713 kB/s eta 0:00:03
     |████████████████████████████████| 69 kB 835 kB/s eta 0:00:01
     |████████████████████████████████| 547 kB 826 kB/s 

In [10]:
#config

NGROK_URL='http://ai_toolkit_elyra_mlflowui_1:5000'
EXPERIMENT='test-mlflow'
EPOCHS=5
VERSION=1

SAVE_PATH='saved_model'
%mkdir -p saved_model

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
#Mlflow settings

import mlflow

#set MLflow server 
mlflow.set_tracking_uri(NGROK_URL)
#Set experiment
if mlflow.get_experiment_by_name(EXPERIMENT) != None:
    exp_id = mlflow.set_experiment(EXPERIMENT)
else: 
    exp_id = mlflow.create_experiment(EXPERIMENT)

#Close active runs
if mlflow.active_run():
    mlflow.end_run()

In [29]:
import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio123'

In [30]:
!mlflow experiments create --experiment-name mlflow-demo-4 --artifact-location s3://mlflowruns/
exp_id = mlflow.set_experiment('mlflow-demo-4')

Traceback (most recent call last):
  File "/opt/conda/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/mlflow/experiments.py", line 45, in create
    exp_id = store.create_expe

In [31]:
import boto3 
with mlflow.start_run(run_id=None, experiment_id=exp_id, run_name=None, nested=False): 
    # log parameters
    mlflow.log_param("epochs", EPOCHS)
    
    
    # Create a features.txt artifact file
    features = "rooms, zipcode, median_price, school_rating, transport"
    with open("features.txt", 'w') as f:
        f.write(features)

    # With artifact_path=None write features.txt under
    # root artifact_uri/artifacts directory
    mlflow.log_artifact("features.txt")
    
    artifact_uri = mlflow.get_artifact_uri()
    print(artifact_uri)
    
    mlflow.end_run()

EndpointConnectionError: Could not connect to the endpoint URL: "http://localhost:9000/mlflowruns/5/e904b1ed02a14a0b86a1ae628d2c9aad/artifacts/features.txt"